### 01-intro Homework Alex Khvatov

In [2]:
#!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

--2025-05-14 12:50:21--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.166.200.107, 3.166.200.185, 3.166.200.219, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.166.200.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47748012 (46M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-02.parquet’

yellow_tripdata_202 100%[===================>]  45.54M  6.07MB/s    in 8.3s    

2025-05-14 12:50:30 (5.50 MB/s) - ‘yellow_tripdata_2023-02.parquet’ saved [47748012/47748012]



In [3]:
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import pickle


from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error


In [5]:
df_jan_data = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')


Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

#### Read the data for January. How many columns are there?

In [6]:
len(df_jan_data.columns)

19

#### Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?


In [7]:
df_jan_data['duration'] = df_jan_data['tpep_dropoff_datetime']-df_jan_data['tpep_pickup_datetime']
df_jan_data.duration = df_jan_data.duration.apply(lambda td: td.total_seconds() / 60)

In [9]:
df_jan_data.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

#### Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?


In [10]:
df_jan_data = df_jan_data[(df_jan_data.duration >=1) & (df_jan_data.duration <=60)]

In [13]:
df_jan_data.duration.describe()

count    3.009173e+06
mean     1.420486e+01
std      9.939386e+00
min      1.000000e+00
25%      7.216667e+00
50%      1.155000e+01
75%      1.818333e+01
max      6.000000e+01
Name: duration, dtype: float64

In [20]:
fraction_left = 3.009173e+06/3.066766e+06
print(f"{fraction_left:.10f}")

0.9812202822


#### Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [21]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [24]:
df_jan_data[categorical]=df_jan_data[categorical].astype(str)

/tmp/ipykernel_781127/633533346.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_data[categorical]=df_jan_data[categorical].astype(str)


In [25]:
train_dicts = df_jan_data[categorical].to_dict(orient='records')

In [26]:
dv=DictVectorizer()
X_train=dv.fit_transform(train_dicts)

In [28]:
X_train.shape

(3009173, 515)

515

#### Question 5. 

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data


In [29]:
target='duration'
y_train = df_jan_data[target].values


In [31]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [32]:
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

/home/alex/.local/share/virtualenvs/mlops-homework-Kw-cColU/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


np.float64(7.649261934850555)

#### Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [33]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >=1) & (df.duration <=60)]
    categorical = ['PULocationID', 'DOLocationID']
    
    df[categorical] = df[categorical].astype(str)

    return df

In [34]:
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

In [35]:
y_val = df_val[target].values

In [36]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [37]:
y_val_pred = lr.predict(X_val)

In [38]:
mean_squared_error(y_val, y_val_pred, squared=False)

/home/alex/.local/share/virtualenvs/mlops-homework-Kw-cColU/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


np.float64(7.811817745843695)